1.1. Загрузите файл password.txt в RDD.

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('password_analysis').getOrCreate()

In [22]:
sc = spark.sparkContext

In [5]:
raw_rdd = spark.sparkContext.textFile('/content/password.txt')

In [6]:
raw_rdd.take(5)

['91981924950', '717893869791585', 'flower2020', 'ikolp2024@', '=aSVka']

In [10]:
raw_rdd.filter(lambda line: line.strip() == '' or line is None).take(5)

[]

1.2. Очистите каждую строку от лишних пробелов и удалите пустые строки, если они есть в файле.

In [7]:
rdd = raw_rdd.filter(
       lambda line: line.strip() != ''
).map(
    lambda line: line.strip()
)

2. Длина паролей:

2.1. Рассчитайте среднюю длину пароля во всем списке. Округлите до целого числа.

2.2. Найдите минимальную и максимальную длину паролей.

2.3. Определите Топ-5 самых распространенных длин паролей (например, сколько паролей имеют длину 8 символов, сколько — 6 и т.д.).

In [12]:
round(rdd.map(
    lambda line: len(line)
).mean())

9

In [13]:
rdd.map(
    lambda line: len(line)
).min()

6

In [14]:
rdd.map(
    lambda line: len(line)
).max()

15

In [18]:
from operator import add


In [25]:
len_freq_rdd = rdd.map(
    lambda line: (len(line), 1)
).reduceByKey(add)

In [27]:
unique_freq = len_freq_rdd.map(
    lambda x: x[-1]
).distinct()

In [19]:
unique_freq.take(5)

[122106, 56936, 115450, 58448, 113429]

In [23]:
sorted_freq = unique_freq.sortBy(lambda x: x).take(5)

In [24]:
broadcast_freq = sc.broadcast(set(sorted_freq))

In [28]:
result = len_freq_rdd.filter(lambda x: x[1] in broadcast_freq.value)


In [30]:
result.map(lambda x: x[0]).collect()

[12, 14, 11, 15, 13]

3. Символьный состав:

3.1. Посчитайте количество паролей, содержащих только цифры.

3.2. Посчитайте количество паролей, содержащих только буквы (как строчные, так и заглавные).

In [31]:
rdd.map(lambda x: x.isdigit()).count()

1000000

In [33]:
from string import ascii_lowercase

In [34]:
al = ascii_lowercase

In [35]:
rdd.map(lambda x: len(set(al).difference(x.lower()) ) == 0 ).count()

1000000

4. Распространенные префиксы/суффиксы:

4.1 Определите Топ-5 самых распространенных префикса длиной 3 символа (например, 123, pas).

4.2 Определите Топ-5 самых распространенных суффикса длиной 3 символа (например, 678, ord).

In [38]:
rdd.map(lambda x: (x[:3], 1) ).reduceByKey(add).sortBy(lambda x: x[-1], ascending = False).top(5)

[('~~s', 1), ('~~c', 1), ('~~V', 1), ('~~T', 1), ('~~M', 1)]

Эталонное решение

In [39]:
rdd_password = sc.textFile("password.txt")
rdd_password = rdd_password.map(lambda x: x.strip())
rdd_password = rdd_password.filter(lambda x: len(x) != 0)

In [40]:
# Средняя дина пароля
len_password = rdd_password.map(lambda x: len(x))
sum_len_password = len_password.sum()
cnt_password = rdd_password.count()
print(f"Средняя длина пароля: {round(sum_len_password/cnt_password)}")

Средняя длина пароля: 9


In [41]:
# Минимальная и максимальна длина
print(f"Минимальная длина пароля: {len_password.min()}")
print(f"Максимальная длина пароля: {len_password.max()}")

# Топ-5 самых распространенных длин паролей
tuple_len = len_password.map(lambda x: (x,1))
cnt_len_password = tuple_len.reduceByKey(lambda a, b: a + b)
for num, cnt in cnt_len_password.top(5, key=lambda x: x[1]):
    print(f"{num}: {cnt}")


Минимальная длина пароля: 6
Максимальная длина пароля: 15
6: 215887
8: 122106
7: 115450
10: 113429
9: 110343


In [42]:
# Количество паролей, содержащих только цифры
only_digit = rdd_password.filter(lambda x: x.isdigit())
print("Количество паролей, содержащих только цифры")
print(only_digit.count())

# Количество паролей, содержащих только буквы
only_alpha = rdd_password.filter(lambda x: x.isalpha())
print("Количество паролей, содержащих только буквы")
print(only_alpha.count())

Количество паролей, содержащих только цифры
441347
Количество паролей, содержащих только буквы
115088


In [43]:
# Топ-5 самых распространенных префикса
list_perfix = rdd_password.map(lambda x: (x[0:3],1))
cnt_perfix = list_perfix.reduceByKey(lambda a, b: a +b)
top_perfix = cnt_perfix.top(5, key = lambda x: x[1])
print("Топ-5 самых распространенных префикса")
for perfix in top_perfix:
    print(perfix)

# Топ-5 самых распространенных суффикса
list_suffix = rdd_password.map(lambda x: (x[-3:],1))
cnt_suffix = list_suffix.reduceByKey(lambda a, b: a +b)
top_suffix = cnt_suffix.top(5, key = lambda x: x[1])
print("Топ-5 самых распространенных суффикса")
for suffix in top_suffix:
    print(suffix)

Топ-5 самых распространенных префикса
('123', 17857)
('qwe', 17054)
('pas', 13290)
('654', 9925)
('edc', 9813)
Топ-5 самых распространенных суффикса
('123', 16135)
('789', 15266)
('020', 14636)
('021', 14558)
('022', 14552)
